# Visualize Predictions

## Parameters

In [ ]:
import torch
import numpy as np

# set random seeds
torch.manual_seed(0)

In [21]:
config = '/home/Talen/foragefish_classifier/configs/exp_resnet18.yaml'
split = 'test'

## Load Data

In [22]:
import sys
sys.path.append('/home/Talen/foragefish_classifier')


In [ ]:
import yaml
from train import create_dataloader, load_model       # NOTE: since we're using these functions across files, it could make sense to put them in e.g. a "util.py" script.

# load config
print(f'Using config "{config}"')
cfg = yaml.safe_load(open(config, 'r'))


# setup entities
dl_test = create_dataloader(cfg, split='test')


# create model

from model import CustomResNet18
import glob

'''
    Creates a model instance and loads the latest model state weights.
'''
model = CustomResNet18(cfg['num_classes'])         # create an object instance of our CustomResNet18 class

# load latest model state

checkpoint = torch.load("/home/Talen/foragefish_classifier/model_states_test/best.pt")
model.load_state_dict(checkpoint['model'])



## Visualize

This is up to you to figure out now. :)

In [24]:
# setup entities
dl_test = create_dataloader(cfg, split='test')
dataset = dl_test.dataset
# filenames = [entry[0] for entry in dataset.data]
# print(filenames)

In [ ]:
from tqdm import trange
import torch.nn.functional as F

device = "cuda"
model.to(device) # puts model weights on to gpu
model.eval() # changes model to eval / inference mode


progressBar = trange(len(dl_test))
pred_all = []
argmax_all = []
# img_list = []
confs_list = []
filename_list = []
gt_all = []
for idx, (data, labels, image_names) in enumerate(dl_test):       # see the last line of file "dataset.py" where we return the image tensor (data) and label

    # put data and labels on device
    data, labels = data.to(device), labels.to(device)

    # forward pass
    prediction = model(data) 
   
    # visualize image that's stored in a batch in variable 'data' (this will be a for loop that iterates a batch)
    # use argmax() over the prediction in a single image, apply it to every image's corresponding prediction.
    
    # Now we use argmax() over the prediction pair of numbers, and apply it to every image's corresponding prediction.
    argmax = prediction.argmax(dim=1)

    # # print(argmax)
    
    # # print(argmax)
    argmax_all.extend(argmax.detach().cpu().numpy())

    # Using softmax to get probabilities
    probabilities = F.softmax(prediction, dim=1)

      # get the predicted labels and their confidence scores
    pred_label = torch.argmax(probabilities, dim=1)
    confidence_scores = torch.max(probabilities, dim=1).values
    confidence_scores = confidence_scores.tolist()
    # print(confidence_scores)

    

    # store the prediction in a list
    # pred_all.append(prediction.detach().cpu().numpy()[0])
    pred_all.extend(prediction.detach().cpu().numpy())
    # img_list.extend(data)
    confs_list.extend(confidence_scores)
    filename_list.extend(image_names)
    gt_all.extend(labels.detach().cpu().numpy())
    
    


# step 1 -visualize predictions + ground truth in matplotlib
# Step 2 - look up weights + biases, how to set them up in the model to log during training
# Step 3 - set up experiments so that when I start a new training run, it generates an experimental folder with the right name 
# copy config file to each experiment folder


In [ ]:
# print(pred_all)
#now we make pred_all a numpy array
pred_all = np.array(pred_all)
print (pred_all.shape)
print (len(confs_list))



In [ ]:
"""
This section of code simply creates a list of confidence scores for each image in the dataset 
by calculating the difference between the highest and second highest prediction values for each image 
(there's only two values right now, but we may have more in the future).
"""

# Now we can use the info in pred_all to calculate a confidence score for each image in the dataset:
# now we'll write the code to do this:
# we'll calculate the confidence score for each image in the dataset.
# confidence score = max(prediction) - second_max(prediction)
# we'll store this in a list called confidence_scores
# confidence_scores = []
# for i in range(len(pred_all)):
#     # we'll use numpy's argsort() function to get the indices of the sorted array
#     sorted_indices = np.argsort(pred_all[i])
#     # we'll get the two highest values from the sorted array
#     highest = sorted_indices[-1]
#     second_highest = sorted_indices[-2]
#     # we'll calculate the confidence score
#     confidence_score = pred_all[i][1] - pred_all[i][0]  # positive for class 1, negative for class 0
#     # we'll store the confidence score in the list
#     confidence_scores.append(confidence_score)

# # now we'll print the confidence scores
# print(confidence_scores)

In [ ]:
# # Now we make a list of our ground-truth labels:
# gt_all = []
# for idx, (data, labels, _) in enumerate(dl_test):
#      gt_all.extend(labels.detach().cpu().numpy())

# # print(gt_all)

# # Now we check the type of gt_all:
# type(gt_all)

In [ ]:
import pandas as pd
import numpy as np

# Convert lists to numpy arrays for consistent handling
# confidence_scores = np.array(confidence_scores)
argmax_all = np.array(argmax_all)
gt_all = np.array(gt_all)

# Create DataFrame with all components
df_combined = pd.DataFrame({
    # 'confidence_scores': confidence_scores.tolist(),  # Convert to list for DataFrame
    'predict_class': argmax_all,
    'ground_truth': gt_all,
    # 'image_id': range(len(img_list)),
    'filenames':filename_list,
    'confs':confs_list
})

# Set image_id as index
# df_combined.set_index('image_id', inplace=True)
#df_combined.set_index('filenames', inplace=True)

# Verify structure
print("DataFrame shape:", df_combined.shape)
print("\nFirst few rows:")
print(df_combined.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Set figure size
plt.figure(figsize=(15, 10))

# Create density plots with filled areas
for gt_class in df_combined['ground_truth'].unique():
    # Get confidence scores for this class
    class_scores = df_combined[df_combined['ground_truth'] == gt_class]['confs']
    
    # Plot density with filled area
    sns.kdeplot(data=class_scores, 
                fill=True,  # Fill area under curve
                alpha=0.5,  # Transparency
                label=f'Class {gt_class}')

# Customize plot
plt.xlabel('Confidence Scores')
plt.ylabel('Density')
plt.title('Distribution of Confidence Scores by Class')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [76]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Create figure with two panels
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Panel 1: ground_truth = 0
data_gt0 = df_combined[df_combined['ground_truth'] == 0]
correct_gt0 = data_gt0[data_gt0['ground_truth'] == data_gt0['predict_class']]['confs']
incorrect_gt0 = data_gt0[data_gt0['ground_truth'] != data_gt0['predict_class']]['confs']

# Panel 2: ground_truth = 1
data_gt1 = df_combined[df_combined['ground_truth'] == 1]
correct_gt1 = data_gt1[data_gt1['ground_truth'] == data_gt1['predict_class']]['confs']
incorrect_gt1 = data_gt1[data_gt1['ground_truth'] != data_gt1['predict_class']]['confs']


In [ ]:
print(data_gt1)

In [ ]:

# sns.kdeplot(data=correct_gt0, fill=True, alpha=0.5, label='Correct', ax=ax1)

sns.displot(data=data_gt1, x = data_gt1['confs'], hue = 'predict_class', fill = True, alpha = 0.5)
# sns.displot(data=incorrect_gt1, label='InCorrect')



In [ ]:

# sns.kdeplot(data=incorrect_gt0, fill=True, alpha=0.5, label='Incorrect', ax=ax1)
ax1.set_title('Ground Truth = 0 (Empty)', fontsize=16)
ax1.set_xlabel('Confidence Scores')
ax1.set_ylabel('Density')
ax1.grid(True, alpha=0.3)
ax1.legend()

sns.kdeplot(data=correct_gt1, fill=True, alpha=0.5, label='Correct', ax=ax2)
sns.kdeplot(data=incorrect_gt1, fill=True, alpha=0.5, label='Incorrect', ax=ax2)
ax2.set_title('Ground Truth = 1 (Forage Fish)', fontsize=16)
ax2.set_xlabel('Confidence Scores')
ax2.set_ylabel('Density')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
print(df_combined.head())

In [ ]:
# In this plot, we will visualize using a histogram instead of density:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# read in results
df = df_combined

# Calculate logits
df["logits"] = np.log(df["confs"] / (1 - df["confs"]))

# Classify as positive or negative
df["label"] = np.where(df["predict_class"] == df["ground_truth"], "Positive", "Negative")

# choose class
chosen_class = 1
filtered_df = df[df["ground_truth"] == chosen_class]

# Plot histogram for the first class
plt.figure(figsize=(8, 6))
sns.histplot(data=filtered_df, x="logits", hue="label", kde=True, bins=10, alpha = 0.5)
plt.title(f"Logit Distribution for Class {chosen_class}")
plt.xlabel("Logits")
plt.ylabel("Frequency")
plt.show()

# Plot histogram for the first class
plt.figure(figsize=(8, 6))
sns.histplot(data=filtered_df, x="confs", hue="label", kde=True, bins=10, alpha = 0.5)
plt.title(f"Confidence Distribution for Class {chosen_class}")
plt.xlabel("Confidence")
plt.ylabel("Frequency")
plt.show()

In [ ]:
print(df_combined)


# Now we visualize the predictions and ground truth in matplotlib

# import matplotlib.pyplot as plt
# import numpy as np

# Let's visualize the first 10 images
# for i in range(10):
#     plt.imshow(data[i].permute(1,2,0))
#     plt.title(f'Ground truth: {gt_all[i]}, Prediction: {argmax_all[i]}')
#     plt.show()

# Now lets visualize all images in the dataset, with their ground truth and predictions:
# for i in range(len(df)):
#     plt.imshow(data[i].permute(1,2,0))
#     plt.title(f'Ground truth: {gt_all[i]}, Prediction: {argmax_all[i]}')
#     plt.show()



# This visualizes one batch of images, but we want to visualize all images in the dataset.

# for i in range(len(df)):
#     plt.imshow(img_list[i].cpu().permute(1,2,0))
#     plt.title(f'Ground truth: {gt_all[i]}, Prediction: {argmax_all[i]}')
#     plt.show()


     

In [55]:
# Now for df_combined, we plot a batch of images listing the ground truth and confidence scores for each images using the 'filenames' column:
import matplotlib.pyplot as plt
import numpy as np



In [56]:
def display_batch(img_list, df_combined, start_idx=0):
    # Create 4x3 grid with adjusted figure size
    fig, axes = plt.subplots(4, 3, figsize=(12, 16))
    axes = axes.ravel()
    
    # Display up to 12 images per batch
    for i in range(12):
        idx = start_idx + i
        if idx >= len(img_list):
            break
            
        # Get data from dataframe
        gt = df_combined.iloc[idx]['ground_truth']
        pred = df_combined.iloc[idx]['predict_class']
        conf = df_combined.iloc[idx]['confs']
        fname = df_combined.iloc[idx]['filenames']
            
        # Display image and labels
        axes[i].imshow(img_list[idx].cpu().permute(1,2,0))
        axes[i].set_title(f'File: {fname}\nGT: {gt}, Pred: {pred}\nConf: {conf:.3f}')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

### NOTE: This function is designed to display a batch of 12 images at a time. ###



In [ ]:
from PIL import Image
def display_images(filename_list, df_combined, num_batches=1, batch_size=12, grid_layout=(4,3)):
    """
    Display specified number of image batches with metadata
    """
    for batch in range(num_batches):
        # Create figure and axes grid
        rows, cols = grid_layout
        fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*4))
        axes = axes.ravel()
        
        # Calculate start index for current batch
        start_idx = batch * batch_size
        
        # Display images in current batch
        for i in range(batch_size):
            idx = start_idx + i
            if idx >= len(filename_list):
                break
            
            
            # Get metadata from dataframe
            # gt = df_combined.iloc[idx]['ground_truth']
            # pred = df_combined.iloc[idx]['predict_class']
            # conf = df_combined.iloc[idx]['confs']
            # fname = df_combined.iloc[idx][1]#'filenames']
            gt = df_combined.iloc[idx]['ground_truth']
            pred = df_combined.iloc[idx]['predict_class']
            print (pred)
            
            conf = df_combined.iloc[idx]['confs']
            print (conf)
            fname = df_combined.iloc[idx]['filenames']
            print (fname)

            
            # Display image and labels
            img = Image.open('/mnt/class_data/group4/talen/combined_imbalanced/eccv_18_all_images_sm/' + fname)
            axes[i].imshow(img)
            
            #axes[i].imshow(fname.cpu().permute(1,2,0))
            axes[i].set_title(f'File: {fname}\nGT: {gt}, Pred: {pred}\nConf: {conf:.3f}')
            axes[i].axis('off')
        
        # Clear unused subplots
        for j in range(i+1, len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.show()

# Usage:
# Show first batch only
data_gt1 = df_combined[df_combined['ground_truth'] == 1]
correct_gt1 = data_gt1[data_gt1['ground_truth'] == data_gt1['predict_class']]
incorrect_gt1 = data_gt1[data_gt1['ground_truth'] != data_gt1['predict_class']]


display_images(incorrect_gt1["filenames"], incorrect_gt1, num_batches=3)

# Show first 3 batches
# display_images(img_list, df_combined, num_batches=3)

In [ ]:
from PIL import Image
def display_images(filename_list, df_combined, num_batches=1, batch_size=12, grid_layout=(4,3)):
    """
    Display specified number of image batches with metadata
    """
    for batch in range(num_batches):
        # Create figure and axes grid
        rows, cols = grid_layout
        fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*4))
        axes = axes.ravel()
        
        # Calculate start index for current batch
        start_idx = batch * batch_size
        
        # Display images in current batch
        for i in range(batch_size):
            idx = start_idx + i
            if idx >= len(filename_list):
                break
            
            
            # Get metadata from dataframe
            # gt = df_combined.iloc[idx]['ground_truth']
            # pred = df_combined.iloc[idx]['predict_class']
            # conf = df_combined.iloc[idx]['confs']
            # fname = df_combined.iloc[idx][1]#'filenames']
            gt = df_combined.iloc[idx]['ground_truth']
            pred = df_combined.iloc[idx]['predict_class']
            print (pred)
            
            conf = df_combined.iloc[idx]['confs']
            print (conf)
            fname = df_combined.iloc[idx]['filenames']
            print (fname)

            
            # Display image and labels
            img = Image.open('/mnt/class_data/group4/talen/combined_imbalanced/eccv_18_all_images_sm/' + fname)
            axes[i].imshow(img)
            
            #axes[i].imshow(fname.cpu().permute(1,2,0))
            axes[i].set_title(f'File: {fname}\nGT: {gt}, Pred: {pred}\nConf: {conf:.3f}')
            axes[i].axis('off')
        
        # Clear unused subplots
        for j in range(i+1, len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.show()

# Usage:
# Show first batch only
data_gt0 = df_combined[df_combined['ground_truth'] == 0]
correct_gt0 = data_gt0[data_gt0['ground_truth'] == data_gt0['predict_class']]
incorrect_gt0 = data_gt0[data_gt0['ground_truth'] != data_gt0['predict_class']]


display_images(incorrect_gt0["filenames"], incorrect_gt0, num_batches=3)

# Show first 3 batches
# display_images(img_list, df_combined, num_batches=3)

In [ ]:
# Jan 23  11:00am - this code will take the lowest confidence correct scores, and highest confidence incorrect scores

def display_confidence_cases(img_list, df_combined, n_images=20, grid_layout=(5,4)):
    """Display lowest confidence correct class 1 and highest confidence incorrect class 1"""
    
    # Filter case 1: Correct class 1 predictions (lowest confidence)
    correct_class1 = df_combined[
        (df_combined['ground_truth'] == 1) & 
        (df_combined['predict_class'] == 1)
    ].sort_values('confs', ascending=True).head(n_images)
    
    # Filter case 2: Incorrect predictions of class 1 (highest confidence)
    incorrect_class1 = df_combined[
        (df_combined['ground_truth'] == 1) & 
        (df_combined['predict_class'] == 0)
    ].sort_values('confs', ascending=False).head(n_images)
    
    # Display both cases
    for case, title in zip([correct_class1, incorrect_class1], 
                          ['Lowest Confidence Correct Class 1', 
                           'Highest Confidence Incorrect Class 1']):
        
        if len(case) == 0:
            print(f"No images found for {title}")
            continue
            
        rows, cols = grid_layout
        fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*4))
        axes = axes.ravel()
        
        n_display = min(len(case), len(axes))
        
        # Display available images
        for i in range(n_display):
            idx = case.index[i]
            row = case.iloc[i]
            
            # Get image and metadata
            gt = row['ground_truth']
            pred = row['predict_class']
            conf = row['confs']
            fname = row[1]  # 'filenames'
            
            # Display image
            axes[i].imshow(img_list[idx].cpu().permute(1,2,0))
            axes[i].set_title(f'File: {fname}\nGT: {gt}, Pred: {pred}\nConf: {conf:.3f}')
            axes[i].axis('off')
        
        # Clear remaining subplots
        for j in range(n_display, len(axes)):
            axes[j].axis('off')
        
        plt.suptitle(title, fontsize=16)
        plt.tight_layout()
        plt.show()

# Usage
display_confidence_cases(img_list, df_combined)

In [ ]:
def display_confidence_cases(img_list, df_combined, n_images=20, grid_layout=(5,4)):
    """Display lowest confidence correct class 1 and highest confidence incorrect class 1"""
    
    # Filter cases
    correct_class1 = df_combined[
        (df_combined['ground_truth'] == 1) & 
        (df_combined['predict_class'] == 1)
    ].sort_values('confs', ascending=True).head(n_images)
    
    incorrect_class1 = df_combined[
        (df_combined['ground_truth'] == 1) & 
        (df_combined['predict_class'] == 0)
    ].sort_values('confs', ascending=False).head(n_images)
    
    # Display both cases
    for case, title in zip([correct_class1, incorrect_class1], 
                          ['Lowest Confidence Correct Class 1', 
                           'Highest Confidence Incorrect Class 1']):
        
        if len(case) == 0:
            print(f"No images found for {title}")
            continue
            
        rows, cols = grid_layout
        # Increase figure height to accommodate title
        fig = plt.figure(figsize=(cols*4, rows*4 + 1))
        
        # Add padding for title
        plt.subplots_adjust(top=0.93)
        
        # Create subplot grid
        gs = fig.add_gridspec(rows, cols)
        axes = [fig.add_subplot(gs[i, j]) for i in range(rows) for j in range(cols)]
        
        n_display = min(len(case), len(axes))
        
        # Display available images
        for i in range(n_display):
            idx = case.index[i]
            row = case.iloc[i]
            
            gt = row['ground_truth']
            pred = row['predict_class']
            conf = row['confs']
            fname = row[1]
            
            axes[i].imshow(img_list[idx].cpu().permute(1,2,0))
            axes[i].set_title(f'File: {fname}\nGT: {gt}, Pred: {pred}\nConf: {conf:.3f}')
            axes[i].axis('off')
        
        # Clear remaining subplots
        for j in range(n_display, len(axes)):
            axes[j].axis('off')
        
        plt.suptitle(title, fontsize=16, y=0.98)
        plt.tight_layout()
        plt.show()

# Usage
display_confidence_cases(img_list, df_combined)